In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers.recurrent import LSTM
from keras.layers import Embedding ,Activation,Input, Concatenate,Add,Merge,Dense, Dropout,Flatten,Reshape,Convolution1D,MaxPooling1D,Convolution2D,MaxPooling2D,TimeDistributed,Lambda
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l2
from keras.optimizers import Adam,SGD,RMSprop,Adadelta
from keras.layers.advanced_activations import LeakyReLU
from IPython.core.display import HTML
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import model_from_json
from keras import backend
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from collections import defaultdict

Using TensorFlow backend.


In [2]:
backend.set_image_dim_ordering('tf')

In [3]:
# coding: shift_jisx0213

In [4]:
COLUMNS = ["year","month","date","kaiji","place","index","race_num","race_name","class_code","track","corse_code","distance","condition",
           "horse_name","horse_sex","horse_age","jockey","load_weight","heads_count","horse_num","finish_order","finish_orderP","ijyou",
           "margin","ninki","time","time_clock","non","lap1","lap2","lap3","lap4","3F","horse_weight","trainer","attach","prize","family_line",
           "jockey_code","trainer_code","race_id","horse_owner","producer","father","mother","m_father","hair_color","birthday","odds",
           "horse_mark","race_mark","PCI"]

COLUMNS2 = ["date","place","race_num","horse_num","race_name","track","distance","horse_name","horse_sex","horse_age","jockey",
            "load_weight","trainer","attach","non1","non2","father","mother","race_id","non6","m_father","non8","non9","non10","non11","non12",
            "non13","non14","non15","non16","non17","non18","non19","non20","non21","non22","non23","non24","non25","non26",
            "non27","non28"]


DCOLUMNS= ["month","date","kaiji","index","class_code","corse_code","finish_orderP","ijyou",
           "margin","ninki","time_clock","non","lap1","lap2","lap3","lap4","3F","prize","family_line",
           "jockey_code","trainer_code","horse_owner","producer","hair_color","birthday","heads_count","condition","horse_weight",
           "horse_mark","race_mark","PCI","m_father","father","mother","heads_count","attach","trainer","race_num"]

DCOLUMNS2 = ["non1","non2","non6","non8","non9","non10","non11","non12","non13",
            "non14","non15","non16","non17","non18","non19","non20","non21","non22","non23","non24","non25","non26",
            "non27","non28","date","m_father","mother","father","attach","trainer","race_num","race_id"]


In [5]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

In [6]:
df_train_all = pd.read_csv('/Users/pp/jupyter/horse_data/DS170604.CSV', names = COLUMNS, engine='python',encoding="shift_jisx0213",
                           converters = {'horse_name' : strip},skiprows = 1)
df_train = df_train_all.drop(DCOLUMNS,axis=1)

In [7]:
df_input_all = pd.read_csv('/Users/pp/jupyter/horse_data/INPUT4.CSV', names = COLUMNS, engine='python',encoding="shift_jisx0213",skiprows = 1)
df_input = df_input_all.drop(DCOLUMNS, axis=1)
df_input = df_input[0:300000]

In [8]:
df_input['odds'] = df_input['odds'].where(df_input['odds'] > 0, 0)

In [9]:
df_prediction_all = pd.read_csv('/Users/pp/jupyter/prediction_data/DE170604.CSV', names = COLUMNS2, engine='python',
                                encoding="shift_jisx0213",converters = {'Description' : strip},skiprows = 1)
df_prediction = df_prediction_all.drop(DCOLUMNS2,axis=1)
df_prediction = df_prediction[df_prediction["race_name"] != u"新馬"]

In [10]:
df_prediction

,place,horse_num,race_name,track,distance,horse_name,horse_sex,horse_age,jockey,load_weight
0,東京,2,未勝利・牝,ダ,1600,ラブリイメッセージ,牝,3,横山武史,51.0
1,東京,3,未勝利・牝,ダ,1600,ハッピーパーティー,牝,3,吉田豊,54.0
2,東京,4,未勝利・牝,ダ,1600,サンボストン,牝,3,野中悠太,51.0
3,東京,5,未勝利・牝,ダ,1600,ノヴァルーナ,牝,3,モレイラ,54.0
4,東京,6,未勝利・牝,ダ,1600,ワカバ,牝,3,井上敏樹,52.0
5,東京,7,未勝利・牝,ダ,1600,ヒスイ,牝,3,柴山雄一,54.0
6,東京,8,未勝利・牝,ダ,1600,チャームクォーク,牝,3,石橋脩,54.0
7,東京,9,未勝利・牝,ダ,1600,コースタルブルー,牝,3,伴啓太,52.0
8,東京,10,未勝利・牝,ダ,1600,クリノユウコリン,牝,3,木幡初也,53.0
9,東京,11,未勝利・牝,ダ,1600,ベルベールダンス,牝,3,丸山元気,54.0


In [11]:
df_input

,year,place,race_name,track,distance,horse_name,horse_sex,horse_age,jockey,load_weight,horse_num,finish_order,time,race_id,odds
0,17,中山,1000万下,ダ,1200,タイセイラナキラ,牝,4,木幡巧也,53.0,16,1,71.2,0617341216,10.8
1,17,中山,1000万下,ダ,1200,ダイワエキスパート,牡,5,石川裕紀,57.0,8,2,71.3,0617341208,15.7
2,17,中山,1000万下,ダ,1200,ヨウライフク,セ,5,大野拓弥,57.0,6,3,71.3,0617341206,5.3
3,17,中山,1000万下,ダ,1200,シゲルヒラマサ,牡,4,武藤雅,54.0,11,4,71.4,0617341211,41.0
4,17,中山,1000万下,ダ,1200,トウショウカウント,牡,5,井上敏樹,55.0,5,5,71.5,0617341205,31.9
5,17,中山,1000万下,ダ,1200,トモトモリバー,牝,4,江田照男,55.0,12,6,71.6,0617341212,7.9
6,17,中山,1000万下,ダ,1200,ワンボーイ,牡,5,田中勝春,57.0,14,7,71.7,0617341214,28.5
7,17,中山,1000万下,ダ,1200,トーセンミッション,牡,5,横山和生,57.0,7,8,71.7,0617341207,30.9
8,17,中山,1000万下,ダ,1200,セクシーボーイ,牡,6,田辺裕信,57.0,13,9,71.8,0617341213,13.5
9,17,中山,1000万下,ダ,1200,エビスリアン,牡,4,菱田裕二,57.0,2,10,71.8,0617341202,11.9


In [12]:
batch_size = 5
race_all = pd.DataFrame()
Fl = ['']*18
Fc = ['']*18
Fa = ['']*18
m=0
for i in range(18):
    Fc[i] = pd.DataFrame()
for i in range(len(df_prediction)):
    name = df_prediction["horse_name"].iloc[i]
    #その馬(name)の過去成績
    records = df_input[df_input["horse_name"]==name]
    if len(records) > batch_size:
        for j in range(batch_size):
            race_name = records["race_id"].iloc[j][:-2]
            #頭がrace_nameとなるカラムを検索
            concat_race = pd.DataFrame()
            for k in range(len(df_input)):
                search = df_input["race_id"].iloc[k].startswith(race_name)
                if search == True:
                    temp = df_input.iloc[k]
                    concat_race = pd.concat([concat_race,temp],axis=1)
            null_df = pd.DataFrame(np.zeros((1,df_input.shape[1])),columns=df_input.columns.values)
            c = concat_race.T
            while(len(c) != 18):
                c = pd.concat([c,null_df],axis=0)
            race_all = pd.concat([race_all,c],axis=0)

for x in range(len(race_all)/18):
    for y in range(18):
        Fl[y] = race_all[m+y:m+y+1]
        Fc[y] = pd.concat([Fc[y],Fl[y]])
        m = m+18                    
for i in range(18):
    Fa[i] = np.array(Fc[i])

In [13]:
cp = race_all.copy()
cp = cp.drop(["year","finish_order","time","odds"], axis =1)
d = defaultdict(LabelEncoder)
data_all = pd.concat([cp,df_prediction],axis=0)
LE = data_all.copy()
LE = LE.drop(["distance","horse_age","horse_num"], axis =1)
# Encoding the variable
Encoded = LE.apply(lambda x: d[x.name].fit_transform(x))
Encoded_all = pd.concat([Encoded,data_all["distance"],data_all["horse_age"],data_all["horse_num"]],axis=1)

train_data = Encoded_all[:len(cp)]
prediction_data = Encoded_all[len(cp):]
# Inverse the encoded
Decoded = Encoded.apply(lambda x: d[x.name].inverse_transform(x))


/Users/pp/anaconda/envs/python2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [14]:
HTML(race_all.head(100).to_html())

,distance,finish_order,horse_age,horse_name,horse_num,horse_sex,jockey,load_weight,odds,place,race_id,race_name,time,track,year
1445,1600,1,3,レッドオルガ,15,牝,和田竜二,54,3.3,中京,0717250515,未勝利,94.3,芝,17
1446,1600,2,3,レディギャング,9,牝,幸英明,54,12.6,中京,0717250509,未勝利,94.4,芝,17
1447,1600,3,3,メイショウミハル,2,牝,城戸義政,52,9.3,中京,0717250502,未勝利,94.7,芝,17
1448,1600,4,3,ブルベアバイソン,4,牡,古川吉洋,56,15.9,中京,0717250504,未勝利,95.1,芝,17
1449,1600,5,3,グラドゥアーレ,12,牡,柴山雄一,56,2.3,中京,0717250512,未勝利,95.3,芝,17
1450,1600,6,3,フィールザフォース,13,牡,丸田恭介,56,48.5,中京,0717250513,未勝利,95.3,芝,17
1451,1600,7,3,レンジャックマン,1,牡,中谷雄太,56,30.7,中京,0717250501,未勝利,95.3,芝,17
1452,1600,8,3,ロードブレイク,8,牡,丸山元気,56,88,中京,0717250508,未勝利,95.5,芝,17
1453,1600,9,3,サヤネエ,5,牝,藤田菜七,51,12.6,中京,0717250505,未勝利,95.6,芝,17
1454,1600,10,3,タマモヒナマツリ,7,牝,富田暁,51,44.6,中京,0717250507,未勝利,95.7,芝,17


In [15]:
Fl = ['']*18
Fc = ['']*18
Fa = ['']*18
m=0
for i in range(18):
    Fc[i] = pd.DataFrame()

for i in range(len(train_data)/18):
    for j in range(18):
        Fl[j] = train_data[m+j:m+j+1]
        Fc[j] = pd.concat([Fc[j],Fl[j]])
    m = m+18
for i in range(18):
    Fa[i] = np.array(Fc[i])

In [11]:
label = ['']*17
for i in range(17):
    label[i] = np.ones((175,1))

In [12]:
label

[array([[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 

In [17]:
vl = ['']*18
vc = ['']*18
v = ['']*18
m=0
for i in range(18):
    vc[i] = pd.DataFrame()

for i in range(len(prediction_data)/18):
    for j in range(18):
        vl[j] = prediction_data[m+j:m+j+1]
        vc[j] = pd.concat([vc[j],vl[j]])
    m = m+18

In [35]:
vv = ['']*18

for i in range(18):
    vv[i] = pd.DataFrame()
    
for i in range(18):
    m = 0
    for j in range(len(vc[0])):
        temp = Fc[i][m:m+4]
        vv[i] = pd.concat([vv[i],vc[i][j:j+1],temp],axis=0)
        m = m+5
        
for i in range(18):
    v[i] = np.array(vv[i])

In [36]:
v[0]

array([[1957, 2, 95, 3, 7, 32, 37, 1, 1600, 3, 2],
       [2038, 2, 42, 6, 1, 1621, 35, 2, 1600, 3, 15],
       [97, 3, 4, 8, 7, 788, 35, 2, 1600, 3, 1],
       [1870, 2, 101, 6, 2, 1033, 38, 1, 1800, 2, 4],
       [1401, 2, 146, 6, 7, 666, 38, 2, 1400, 2, 8],
       [619, 3, 30, 8, 7, 28, 35, 1, 1300, 3, 4],
       [2126, 2, 85, 6, 2, 1290, 35, 1, 1200, 3, 5],
       [921, 3, 93, 8, 7, 884, 35, 1, 1400, 3, 4],
       [392, 2, 120, 3, 1, 1535, 35, 1, 1400, 3, 6],
       [606, 2, 41, 6, 2, 1099, 35, 1, 1200, 3, 1],
       [1188, 2, 85, 6, 7, 2, 35, 2, 1400, 3, 6],
       [2007, 3, 51, 8, 2, 1316, 35, 1, 1800, 3, 15],
       [1223, 3, 99, 8, 2, 1224, 35, 1, 1800, 3, 6],
       [1582, 3, 40, 8, 2, 1153, 35, 1, 1800, 3, 10],
       [1973, 3, 34, 8, 1, 1520, 35, 1, 1400, 3, 7],
       [1779, 2, 14, 6, 7, 10, 35, 2, 2400, 3, 6],
       [802, 2, 40, 7, 7, 768, 1, 1, 1400, 5, 11],
       [1780, 2, 102, 6, 9, 2267, 46, 2, 1400, 4, 7],
       [48, 2, 72, 6, 7, 741, 2, 2, 1400, 3, 2],
       [214

In [5]:
nb_filter = 32
Input_length = 11
embedding_size = 50
batch_size = 5

In [6]:
d1 = Dense(20,activation='relu')
d2 = Dense(10,activation='relu')
e = Embedding(output_dim=embedding_size, input_dim=10000, input_length=Input_length)
c = Convolution1D(nb_filter = 2, filter_length=2, border_mode = 'same', activation='relu')
lstm = LSTM(batch_size*18)
s = Dense(1)
F = ['']*18
em = ['']*18
c1 = ['']*18
r1 = ['']*18
f = ['']*18
F_d1 = ['']*18
F_d2 = ['']*18
score = ['']*18
negated_score = ['']*18
diff = ['']*17
temp = ['']*18
diff_all = ['']*18
prob = ['']*18

for i in range(18):
    F[i] = Input(shape = (Input_length, ), dtype = "float64")
    em[i] = e(F[i])
    c1[i] = c(em[i])
    r1[i] = lstm(c1[i])
    F_d1[i] = d1(r1[i])
    F_d2[i] = d2(F_d1[i])
    score[i] = s(F_d2[i])
    negated_score[i] = Lambda(lambda x: -1 * x, output_shape = (1, ))(score[i])

for n in range(1,17):
    diff[n] = Add()([score[0], negated_score[n]])

for i in range(1,17):
    prob[i] = Activation("sigmoid")(diff[i])

# Build model.
model_rank = Model(inputs = [F[k] for k in range(18)], 
                   outputs = [prob[k] for k in range(1,16)])
adam = Adam(lr=0.001)
model_rank.compile(optimizer = 'adadelta', loss = "binary_crossentropy")

/Users/pp/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", padding="same", filters=2, kernel_size=2)`


In [9]:
prob[k]

<tf.Tensor 'activation_15/Sigmoid:0' shape=(?, 1) dtype=float32>

In [24]:
model_rank.fit( [Fa[k] for k in range(18)],[label[k] for k in range(1,16)], batch_size = batch_size, epochs = 50, shuffle = False)

Epoch 1/50
175/175 [==============================] - 34s - loss: 10.3969 - activation_17_loss: 0.6932 - activation_18_loss: 0.6932 - activation_19_loss: 0.6932 - activation_20_loss: 0.6932 - activation_21_loss: 0.6932 - activation_22_loss: 0.6932 - activation_23_loss: 0.6932 - activation_24_loss: 0.6931 - activation_25_loss: 0.6931 - activation_26_loss: 0.6931 - activation_27_loss: 0.6931 - activation_28_loss: 0.6931 - activation_29_loss: 0.6931 - activation_30_loss: 0.6931 - activation_31_loss: 0.6930    
Epoch 2/50
175/175 [==============================] - 12s - loss: 10.3856 - activation_17_loss: 0.6931 - activation_18_loss: 0.6931 - activation_19_loss: 0.6931 - activation_20_loss: 0.6931 - activation_21_loss: 0.6931 - activation_22_loss: 0.6931 - activation_23_loss: 0.6931 - activation_24_loss: 0.6932 - activation_25_loss: 0.6930 - activation_26_loss: 0.6929 - activation_27_loss: 0.6926 - activation_28_loss: 0.6918 - activation_29_loss: 0.6907 - activation_30_loss: 0.6902 - activ

In [41]:
filepath = 'weights.best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
#surrarize results for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = "upper_left")

f_name = '/Users/pp/jupyter/graph/DNN_binary_acc.png'
plt.savefig(f_name)
plt.show()

In [ ]:
#summarize results for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper_left')

f_name = '/Users/pp/jupyter/graph/DNN_binary_loss.png'
plt.savefig(f_name)
plt.show()

In [ ]:
model.summary()

In [ ]:
model.save_weights('/Users/pp/jupyter/train_data_cnn_a_horseTree.h5')

In [37]:
prediction2 = model_rank.predict([v[k] for k in range(18)],batch_size=batch_size)

In [38]:
pd.Series(prediction2)

0     [[0.00228206], [1.0], [1.0], [1.0], [1.0], [1....
1     [[0.584715], [1.0], [1.0], [1.0], [1.0], [0.44...
2     [[7.33827e-05], [1.0], [1.0], [1.0], [1.0], [0...
3     [[0.339138], [1.0], [1.0], [1.0], [1.0], [0.62...
4     [[0.0180455], [1.0], [1.0], [1.0], [1.0], [0.5...
5     [[0.344006], [1.0], [1.0], [1.0], [0.999998], ...
6     [[0.459559], [1.0], [0.999999], [1.0], [1.0], ...
7     [[0.314086], [1.0], [1.0], [1.0], [1.0], [0.33...
8     [[0.458313], [1.0], [1.0], [1.0], [1.0], [0.66...
9     [[0.114468], [1.0], [1.0], [1.0], [1.0], [0.32...
10    [[0.00302041], [1.0], [1.0], [1.0], [1.0], [0....
11    [[9.82444e-08], [1.0], [1.0], [1.0], [1.0], [0...
12    [[0.750014], [1.0], [1.0], [1.0], [1.0], [0.89...
13    [[0.527786], [1.0], [1.0], [1.0], [1.0], [0.91...
14    [[0.22261], [1.0], [1.0], [1.0], [1.0], [0.000...
dtype: object

In [39]:
get_score = backend.function([F[k] for k in range(18)],
                             [score[k] for k in range(18)])

In [40]:
sc = get_score([v[k] for k in range(18)])
sca = np.array(sc)
scd = pd.DataFrame(sca.reshape((18,len(v[0]))))

In [52]:
scd.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-10.009285,-3.928893,-10.351443,-0.489536,-9.342147,-6.012634,-9.363794,-9.847165,-9.228200,-9.842149,-7.963387,-4.209947,6.126523,-11.107970,-10.120543,-8.758765,-10.060423,-8.863512
1,25.393127,-5.826447,-10.137809,-7.729074,5.287224,-9.047049,-8.708352,-8.574010,-9.705026,-9.658822,7.733027,-9.624385,-9.940756,-11.482379,-11.220841,2.574307,-13.303889,-13.303889
2,14.838363,-1.586962,-9.010155,-1.962498,-8.425561,-1.149171,-4.477601,0.775064,-8.299382,-6.009904,-9.768287,-10.172280,-9.674852,-10.463786,-7.441163,-9.656301,-13.264106,-13.321410
3,11.359753,-9.804699,-10.176608,-11.545064,-10.573522,-5.429761,-10.510267,-6.136204,-11.015502,-8.855213,-11.110034,-6.664753,-10.565410,-4.832615,-9.512565,-11.393003,-13.244680,-13.244680
4,20.981207,-10.475806,-8.119816,-10.115480,-10.688616,-9.744340,7.624705,5.659464,-7.536738,-6.701776,-7.866420,-10.673348,-11.245630,-10.953343,-2.680137,-9.877941,-13.253623,-13.253623
5,-8.876361,15.990305,-8.665268,-8.894718,-9.390847,-8.977174,-10.200294,-10.490994,-8.194949,-9.562845,-8.165929,-0.579380,-10.559484,-10.967421,-11.293196,-1.228243,-10.634053,-10.703157
6,23.272383,-9.067261,-1.160241,-10.652587,-10.469664,4.263797,8.747545,-9.300781,-4.052077,-11.213911,-9.243108,-11.440515,-11.116531,-9.224777,-10.371390,-9.844505,-13.257040,-13.257040
7,23.977791,4.984442,-4.759733,-8.806293,10.200836,-8.940791,-10.160754,-4.405479,3.214711,-8.200765,-9.974115,-10.503299,-10.001125,-10.212523,-9.678146,-6.862800,-13.274853,-13.253623
8,22.613754,-1.432200,-10.878884,-9.182443,-9.982768,-8.660454,-3.598548,-8.671448,-9.357326,-8.890543,-10.727308,-4.268562,-10.394624,-9.301730,-10.124031,-10.345638,-13.288056,-13.288056
9,17.045521,-8.506854,-9.858700,3.789960,-9.851832,-8.695564,-10.451688,-9.658709,-10.541832,-9.293219,-10.432511,-5.578463,-10.465347,-9.932055,-10.800305,-8.790118,-13.264472,-13.264472


In [43]:
df_results = pd.read_csv("/Users/pp/jupyter/results/20170604.csv", engine='python',encoding="shift_jisx0213")

In [44]:
results = pd.concat([df_results[u"レース名"],df_results[u"馬名"],df_results[u"馬番"],df_results[u"着順"]],axis=1)

In [45]:
HTML(results.head(100).to_html())

,レース名,馬名,馬番,着順
0,三浦特別1000,シュナウザー,14,１
1,三浦特別1000,アメリカンストーム,10,２
2,三浦特別1000,ラバニーユ,12,３
3,三浦特別1000,シーリーヴェール,11,４
4,三浦特別1000,ジェイケイライアン,5,５
5,三浦特別1000,ヴァッハウ,6,６
6,三浦特別1000,アルファアリエス,8,７
7,三浦特別1000,ブルーボサノヴァ,1,８
8,三浦特別1000,ダイチヴュルデ,15,９
9,三浦特別1000,ゴールドリーガル,16,10


In [50]:
rl = ['']*18
rc = ['']*18

m=0
for i in range(18):
    rc[i] = pd.DataFrame()

for i in range(len(df_prediction)/18):
    for j in range(18):
        rl[j] = df_prediction[m+j:m+j+1]
        rc[j] = pd.concat([rc[j],rl[j]],axis=0)
    m = m+18

In [51]:
rc

[    place  horse_num race_name track  distance horse_name horse_sex  \
 0      東京          2     未勝利・牝     ダ      1600  ラブリイメッセージ         牝   
 18     東京          4       未勝利     ダ      1300   コンピレーション         牡   
 36     東京          6       未勝利     芝      1400  デルマアンコツバキ         牝   
 54     東京          6       未勝利     芝      2400   ムスターヴェルク         牝   
 88     東京          6       未勝利     ダ      1400   データヴァリュー         牡   
 106    東京          8     500万下     ダ      2100   セイカエドミザカ         牡   
 124    東京         10   500万下・牝     ダ      1400    アデレードヒル         牝   
 142    東京          2  由比ヶＨ1000     芝      1400   タケデンタイガー         牡   
 160    東京          6    安田記念G1     芝      1600   レッドファルクス         牡   
 178    東京          6  三浦特別1000     ダ      1600      ヴァッハウ         牡   
 196    阪神          8     未勝利・牝     ダ      1400        ピカロ         牝   
 214    阪神         10       未勝利     ダ      1200    シゲルドラネコ         牡   
 232    阪神         12       未勝利     ダ      1800  クリスティヴィッツ      